In [1]:
# データ加工・処理・分析ライブラリ
import numpy as np
import numpy.random as random
import scipy as sp
from pandas import Series, DataFrame
import pandas as pd

# 可視化ライブラリ
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

# 機械学習ライブラリ
import sklearn

# 小数第三位まで表示
%precision 3

'%.3f'

<span style="font-size: 200%">重回帰</span>

In [2]:
# インポート
import requests, zipfile
import io

# 自動車価格データを取得
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data'
res = requests.get(url).content

# 取得したデータをDataFrameオブジェクトとして読み込み
auto_df = pd.read_csv(io.StringIO(res.decode('utf-8')), header=None)

# データの列にラベルを設定
auto_df.columns = ['symboling', 'normalized-losses', 'make', 'fuel-type', 'aspiration', 
                   'num-of-doors', 'body-style', 'drive-wheels', 'engine-location', 'wheel-base', 
                   'length', 'width', 'height', 'curb-weight', 'engine-type', 
                   'num-of-cylinders', 'engine-size', 'fuel-system', 'bore', 'stroke', 
                   'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 
                   'price']


In [3]:
auto_df.head(6)

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450
5,2,?,audi,gas,std,two,sedan,fwd,front,99.8,...,136,mpfi,3.19,3.40,8.5,110,5500,19,25,15250


**データ整理**

In [4]:
# それぞれのカラムに？が何個あるかをカウント
auto = auto_df[['price', 'horsepower', 'width', 'height']]
auto.isin(['?']).sum()

price         4
horsepower    2
width         0
height        0
dtype: int64

In [5]:
'''
不適切なデータの削除
'''
# ?をNaNに置換して、NaNがある行を削除
auto = auto.replace('?', np.nan).dropna()
print('データ型の確認（型変換前）\n{}\n'.format(auto.shape))

データ型の確認（型変換前）
(199, 4)



In [6]:
'''
型の変換
'''
auto = auto.assign(price=pd.to_numeric(auto.price))
auto = auto.assign(horsepower=pd.to_numeric(auto.horsepower))
print('データ型の確認（型変換前）\n{}\n'.format(auto.dtypes))

データ型の確認（型変換前）
price           int64
horsepower      int64
width         float64
height        float64
dtype: object



In [7]:
'''
相関の確認
'''
auto.corr()

,price,horsepower,width,height
price,1.000000,0.810533,0.753871,0.134990
horsepower,0.810533,1.000000,0.615315,-0.087407
width,0.753871,0.615315,1.000000,0.309223
height,0.134990,-0.087407,0.309223,1.000000


**モデル構築と評価**

In [8]:
# データ分割（訓練データとテストデータ）のためのインポート
from sklearn.model_selection import train_test_split

# 重回帰のモデル構築のためのインポート
from sklearn.linear_model import LinearRegression

# 目的変数にpriceを指定、説明変数にそれ以外を指定
X = auto.drop('price', axis=1)
y = auto['price']

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# 重回帰クラスの初期化と学習
model = LinearRegression()
model.fit(X_train, y_train)

# 決定係数を表示
print('決定係数(train):{:.3f}'.format(model.score(X_train, y_train)))
print('決定係数(test):{:.3f}'.format(model.score(X_test, y_test)))

# 回帰係数と切片を表示
print('\n回帰係数\n{}'.format(pd.Series(model.coef_, index=X.columns)))
print('切片:{:.3f}'.format(model.intercept_))


決定係数(train):0.733
決定係数(test):0.737

回帰係数
horsepower      81.651078
width         1829.174506
height         229.510077
dtype: float64
切片:-128409.046


以下は追加課題

**変数増加法(前進的選択法、forward selection method)**

最初に全変数のうち判別に寄与する割合が最大のものを取り込む。  
そしてそれ以後は、すでに取り込まれた変数との共有部分を除いて、  
単独で判別に寄与する割合が最大の変数を取り込んでいく。   
これは、その変数を取り込んで判別関数を計算した時、  
判別効率の増加が最大になることを意味している。    
その際、判別効率の増加があらかじめ決めておいた値、  
すなわち取り込み基準未満になった時に変数の取り込みを終了する。  

**変数減少法(後退的選択法、backward selection method)**

最初に全変数を取り込み、以後は取り込まれた変数のうち  
単独で判別に寄与している割合が最小のものを追い出していく。   
これは、その変数を追い出して判別関数を計算した時、  
判別効率の減少が最小になることを意味している。  
この際、判別効率の減少があらかじめ決めておいた値、  
すなわち追い出し基準以上になった時に変数の追い出しを終了する。

**ステップワイズ法（変数増減法）(stepwise forward selection method)**

増加法と減少法を組み合わせた手法。  
まず増加法と同様に取り込み基準に従って変数を取り込む。  
そして1つ変数を取り込むたびに、すでに取り込んだものの中で  
減少法と同様の追い出し基準を満足するものがないか調べ、  
あればそれを追い出してから、さらに変数の取り込みを続ける。  
この手法では取り込むべき変数も追い出すべき変数もなくなった時に変数の選択を終了する。  
この手法は判別に寄与する割合が大きい変数を少数選択するという特徴を持ち、  
逐次変数選択法の中で最も多用されている。

In [10]:
'''
練習問題8-1
'''

# インポート
import requests, zipfile
import io

# 自動車価格データを取得
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data'
res = requests.get(url).content

# 取得したデータをDataFrameオブジェクトとして読み込み
auto_df = pd.read_csv(io.StringIO(res.decode('utf-8')), header=None)

# データの列にラベルを設定
auto_df.columns = ['symboling', 'normalized-losses', 'make', 'fuel-type', 'aspiration', 
                   'num-of-doors', 'body-style', 'drive-wheels', 'engine-location', 'wheel-base', 
                   'length', 'width', 'height', 'curb-weight', 'engine-type', 
                   'num-of-cylinders', 'engine-size', 'fuel-system', 'bore', 'stroke', 
                   'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 
                   'price']

auto = auto_df[['price', 'length', 'engine-size']]

# ?をNaNに置換して、NaNがある行を削除
auto = auto.replace('?', np.nan).dropna()


In [14]:
# 型の変換
auto = auto.assign(price=pd.to_numeric(auto.price))
auto = auto.assign(length=pd.to_numeric(auto.length))
auto['engine-size'] = auto['engine-size'].astype(int)

In [15]:
auto.corr()

,price,length,engine-size
price,1.000000,0.690628,0.872335
length,0.690628,1.000000,0.685025
engine-size,0.872335,0.685025,1.000000


In [16]:
# データ分割（訓練データとテストデータ）のためのインポート
from sklearn.model_selection import train_test_split

# 重回帰のモデル構築のためのインポート
from sklearn.linear_model import LinearRegression

# 目的変数にpriceを指定、説明変数にそれ以外を指定
X = auto.drop('price', axis=1)
y = auto['price']

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# 重回帰クラスの初期化と学習
model = LinearRegression()
model.fit(X_train, y_train)

# 決定係数を表示
print('決定係数(train):{:.3f}'.format(model.score(X_train, y_train)))
print('決定係数(test):{:.3f}'.format(model.score(X_test, y_test)))

# 回帰係数と切片を表示
print('\n回帰係数\n{}'.format(pd.Series(model.coef_, index=X.columns)))
print('切片:{:.3f}'.format(model.intercept_))


決定係数(train):0.771
決定係数(test):0.763

回帰係数
length         179.544547
engine-size    120.273905
dtype: float64
切片:-33590.411


In [17]:
auto_df.head(3)

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500


In [33]:
auto = auto_df[['price', 'wheel-base', 'bore', 'stroke', 'compression-ratio', 'city-mpg', 'highway-mpg']]
auto = auto.replace('?', np.nan).dropna()

auto['price'] = auto['price'].astype(float).astype(int)
auto['bore'] = auto['bore'].astype(float)
auto['stroke'] = auto['stroke'].astype(float)

In [34]:
auto.head(2)

,price,wheel-base,bore,stroke,compression-ratio,city-mpg,highway-mpg
0,13495,88.6,3.47,2.68,9.0,21,27
1,16500,88.6,3.47,2.68,9.0,21,27


In [35]:
auto.corr()

,price,wheel-base,bore,stroke,compression-ratio,city-mpg,highway-mpg
price,1.000000,0.586661,0.543436,0.082310,0.071079,-0.699738,-0.714941
wheel-base,0.586661,1.000000,0.494884,0.158502,0.249006,-0.496525,-0.565861
bore,0.543436,0.494884,1.000000,-0.055390,0.001250,-0.592655,-0.599218
stroke,0.082310,0.158502,-0.055390,1.000000,0.187923,-0.034696,-0.035201
compression-ratio,0.071079,0.249006,0.001250,0.187923,1.000000,0.332343,0.267668
city-mpg,-0.699738,-0.496525,-0.592655,-0.034696,0.332343,1.000000,0.971577
highway-mpg,-0.714941,-0.565861,-0.599218,-0.035201,0.267668,0.971577,1.000000


In [36]:
# データ分割（訓練データとテストデータ）のためのインポート
from sklearn.model_selection import train_test_split

# 重回帰のモデル構築のためのインポート
from sklearn.linear_model import LinearRegression

# 目的変数にpriceを指定、説明変数にそれ以外を指定
X = auto.drop('price', axis=1)
y = auto['price']

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# 重回帰クラスの初期化と学習
model = LinearRegression()
model.fit(X_train, y_train)

# 決定係数を表示
print('決定係数(train):{:.3f}'.format(model.score(X_train, y_train)))
print('決定係数(test):{:.3f}'.format(model.score(X_test, y_test)))

# 回帰係数と切片を表示
print('\n回帰係数\n{}'.format(pd.Series(model.coef_, index=X.columns)))
print('切片:{:.3f}'.format(model.intercept_))


決定係数(train):0.537
決定係数(test):0.583

回帰係数
wheel-base            -46.505610
bore                 5028.093190
stroke              -2828.789824
compression-ratio     395.347490
city-mpg             -476.151606
highway-mpg          -297.227198
dtype: float64
切片:27506.604
